In [70]:
import numpy as np
import pandas as pd

import csv

In [71]:
df_emdat = pd.read_csv('./datasets/emdat_cleaned.csv')
df_ibtracs = pd.read_csv('./datasets/df_IBTrACS.csv')

In [72]:
#number of records
print('numer of records in the dataset emdat:',len(df_emdat))
print('numer of records in the dataset ibtracs:',len(df_ibtracs))

numer of records in the dataset emdat: 3463
numer of records in the dataset ibtracs: 66922


In [73]:
#columns
df_emdat.columns

Index(['Unnamed: 0', 'Year', 'Seq', 'ISO', 'Group', 'Subgroup', 'Type',
       'Subtype', 'Subsubtype', 'Event name', 'Entry criteria', 'Country name',
       'Region', 'Continent', 'EU member', 'Location', 'Origin',
       'Associated disater', 'Associated disater 2', 'OFDA response', 'Appeal',
       'Declaration', 'Aid contribution ('000$)', 'Magnitude (value)',
       'Magnitude (scale)', 'Latitude', 'Longitude', 'Local time',
       'River basin', 'Start year', 'Start month', 'Start day', 'End year',
       'End month', 'End day', 'Total deaths', 'Num injured', 'Num affected',
       'Num homeless', 'Total affected', 'Reconstruction cost ('000$)',
       'Insured losses ('000$)', 'Total damage ('000$)', 'CPI'],
      dtype='object')

In [74]:
df_ibtracs.columns

Index(['Unnamed: 0', 'SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME',
       'ISO', 'ISO_TIME', 'TRACK', 'NATURE', 'WMO_WIND', 'PRES_CALC',
       'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'STORM_SPD', 'STORM_DR'],
      dtype='object')

# Function

In [75]:
def cleaning_names(df, column_name, print_results=False):
    
    print("Before changing to None")
    
    for index, row in df.iterrows():
        if row[column_name] == "NOT_NAMED":
            df.at[index, column_name] = None
    
    print("DONE: Changing to None")
    
    print("Before excluding words")
    EXCLUDED_WORDS=[ "HURRICANE", "TROPICAL", "TOPICAL", "TYHOON", "DEPRESSION", "SUB-TROPICAL", "STORM", "EX-TROPICAL", "CYCLONE", "TYPHHON", "CYLONE", "TYPHOON", '"', '\'' ]

    for index, row in df.iterrows():
        if pd.notna(row[column_name]):
            for word in EXCLUDED_WORDS:
                if word in row[column_name].upper():
                    print(word)
                    strr = df.at[index, column_name].upper()
                    df.at[index, column_name] = strr.replace(word, '')
    print("DONE: Excluding words")
    print("Before excluding extra words")
    EXCLUDED_EXTRA_WORDS = [ " AND ", "THE ", "/", "(", ")"] 

    for index, row in df.iterrows():
        if pd.notna(row[column_name]):
            for word in EXCLUDED_EXTRA_WORDS:
                if word in row[column_name].upper():
                    print(word)
                    strr = df.at[index, column_name].upper()
                    df.at[index, column_name] = strr.replace(word, ' ')

    print("DONE: Excluding extre words")                
    print("Before removing spaces at the beginning of string")
    for index, row in df.iterrows():
        if pd.notna(row[column_name]):
            nem = row[column_name].upper()
            try:
                while nem[0].isspace():
                    nem = nem[1:]
                    df.at[index, column_name] = nem
                    print("space removed")
            except:
                pass
    print("DONE: Removing spaces at the beginning of string")
    print("Before spliting by characters")

    CHARACTERS = [ " ", ", ", "& "]
    
    for index, row in df.iterrows():
        if pd.notna(row[column_name]):
            for char in CHARACTERS:
                if char in row[column_name].upper():
                    new_name = ""
                    words = row[column_name].split(char)
                    for temp in words:
                        if temp.isalpha() and len(temp)>2:
                            if new_name == "":
                                new_name = new_name + temp
                            else:
                                new_name = new_name + "_" + temp
                        df.at[index, column_name] = new_name
                        
    print("DONE: Spliting by characters")     
    
    if print_results == True:
        print("Just printing results")
        for index, row in df.iterrows():
            if pd.notna(row[column_name]):
                try:
                    nem = row[column_name].upper()
                    for lett in nem:
                        if lett.isdigit():
                            nem = nem.replace(lett, "")
                    if len(nem)>2:
                        df.at[index, column_name] = nem
                        print(nem)
                    else:
                        df.at[index, column_name] = None
                except:
                    pass
    
    print("DONE")
            
    return df

# Emdat

In [76]:
df_emdat.head()

,Unnamed: 0,Year,Seq,ISO,Group,Subgroup,Type,Subtype,Subsubtype,Event name,...,End day,Total deaths,Num injured,Num affected,Num homeless,Total affected,Reconstruction cost ('000$),Insured losses ('000$),Total damage ('000$),CPI
0,12,1902,5,MMR,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,...,4.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.391845
1,15,1903,8,JAM,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,...,10.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
2,18,1904,3,BGD,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300
3,27,1906,15,HKG,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,...,8.0,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,20000.0,3.522300
4,50,1909,10,BGD,Natural,Meteorological,Storm,Tropical cyclone,NaN,NaN,...,15.0,172.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.522300


In [ ]:
df_emdat = cleaning_names(df_emdat, "Event name")
df_emdat.head()

In [78]:
df_emdat.to_csv("emdat_names.csv")

# IBTrACS

In [79]:
df_ibtracs.head()

,Unnamed: 0,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO,ISO_TIME,TRACK,NATURE,WMO_WIND,PRES_CALC,TRACK_TYPE,DIST2LAND,LANDFALL,STORM_SPD,STORM_DR
0,0,1842298N11080,1842,1,NI,BB,NOT_NAMED,IND,1842-10-25 06:00:00,"[[10.87, 79.83], [10.84, 79.35], [10.82, 78.88...",NR,NaN,NaN,main,0,0,9,267
1,1,1842298N11080,1842,1,NI,BB,NOT_NAMED,IND,1842-10-25 09:00:00,"[[10.87, 79.83], [10.84, 79.35], [10.82, 78.88...",NR,NaN,NaN,main,0,0,9,267
2,2,1842298N11080,1842,1,NI,BB,NOT_NAMED,IND,1842-10-25 12:00:00,"[[10.87, 79.83], [10.84, 79.35], [10.82, 78.88...",NR,NaN,NaN,main,0,0,9,267
3,3,1842298N11080,1842,1,NI,BB,NOT_NAMED,IND,1842-10-25 15:00:00,"[[10.87, 79.83], [10.84, 79.35], [10.82, 78.88...",NR,NaN,NaN,main,0,0,9,268
4,4,1842298N11080,1842,1,NI,AS,NOT_NAMED,IND,1842-10-25 18:00:00,"[[10.87, 79.83], [10.84, 79.35], [10.82, 78.88...",NR,NaN,NaN,main,0,0,10,269


In [ ]:
df_ibtracs = cleaning_names(df_ibtracs, "NAME", print_results=True)
df_ibtracs.head()

In [81]:
df_ibtracs.to_csv("emdat_ibtracs.csv")